In [33]:
import numpy as np
import tensorflow as tf
import os
import warnings
warnings.filterwarnings("ignore")

# Sigmoid example

In [6]:
@tf.function
def layer(x, W, b):
    # building the graph
    return tf.math.sigmoid(tf.matmul(x, W) + b)

In [7]:
x = np.array([[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]], dtype=np.float32)

init_w = tf.initializers.RandomUniform(minval=-.1, maxval=.1)(shape=[10,5])
W = tf.Variable(init_w, dtype=tf.float32, name='W')

init_b = tf.initializers.RandomUniform()(shape=[5])
b = tf.Variable(init_b, dtype=tf.float32, name='b')

h = layer(x, W, b)

In [8]:
print(f"h: {h.numpy()}")

h: [[0.5018517  0.49334908 0.50805944 0.44643217 0.46883595]]


# Defining Inputs
3 different way to feed data to tf program:
- as np array
- as tf tensors
- using `tf.data` **API** to create an input *pipeline*



In [10]:
# feeding data as tensor
x = tf.constant(
    [[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]], 
    dtype=tf.float32,
    name="x"
)

init_w = tf.initializers.RandomUniform(minval=-.1, maxval=.1)(shape=[10,5])
W = tf.Variable(init_w, dtype=tf.float32, name='W')

init_b = tf.initializers.RandomUniform()(shape=[5])
b = tf.Variable(init_b, dtype=tf.float32, name='b')

h = layer(x, W, b)

In [11]:
print(f"h: {h.numpy()}")
print(f"type: {type(h)}")

h: [[0.44630384 0.49760556 0.48088944 0.5210194  0.5187529 ]]
type: <class 'tensorflow.python.framework.ops.EagerTensor'>


In [29]:
# tf.data api
filenames = [f"./iris.data.{i}" for i in range(1,4)]

dataset = tf.data.experimental.CsvDataset(
    filenames,
    [
        tf.float32,
        tf.float32,
        tf.float32,
        tf.float32,
        tf.string,
    ]
)

# organize our data into inputs and labels
dataset = dataset.map(lambda x1,x2,x3,x4,y: (tf.stack([x1,x2,x3,x4]), y))

In [30]:
for next_element in dataset:
    x, y = next_element[0].numpy(), next_element[1].numpy().decode("ascii")
    if np.min(x) < .0:
        print(f"(corrupted) X => {x}\tY => {y}")

(corrupted) X => [ 5.8 -4.   1.2  0.2]	Y => Iris-setosa
(corrupted) X => [ 4.6  3.6 -1.   0.2]	Y => Iris-setosa
(corrupted) X => [ 4.7 -3.2  1.6  0.2]	Y => Iris-setosa
(corrupted) X => [-5.   3.5  1.3  0.3]	Y => Iris-setosa
(corrupted) X => [-5.1  3.8  1.6  0.2]	Y => Iris-setosa
(corrupted) X => [ 6.9 -3.1  4.9  1.5]	Y => Iris-versicolor
(corrupted) X => [-5.2  2.7  3.9  1.4]	Y => Iris-versicolor
(corrupted) X => [ 5.8  2.7  4.1 -1. ]	Y => Iris-versicolor
(corrupted) X => [ 6.1 -2.8  4.7  1.2]	Y => Iris-versicolor
(corrupted) X => [ 5.4  3.  -4.5  1.5]	Y => Iris-versicolor
(corrupted) X => [ 7.1 -3.   5.9  2.1]	Y => Iris-virginica
(corrupted) X => [ 5.7  2.5  5.  -2. ]	Y => Iris-virginica
(corrupted) X => [-6.   2.2  5.   1.5]	Y => Iris-virginica
(corrupted) X => [ 7.2 -3.2  6.   1.8]	Y => Iris-virginica
(corrupted) X => [ 6.1  2.6  5.6 -1.4]	Y => Iris-virginica


In [31]:
dataset = dataset.filter(lambda x, y: tf.reduce_min(x) > 0)

In [32]:
batch_size = 5
dataset = dataset.batch(batch_size=batch_size)
for next_element in dataset:
    x, y = next_element[0].numpy(), next_element[1].numpy()    
    print(f"x.shape = {x.shape}, y.shape = {y.shape}")
    break

x.shape = (5, 4), y.shape = (5,)


# Mnist classification

In [34]:
os.makedirs("data", exist_ok=True)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path=os.path.join(os.getcwd(), 'data', 'mnist.npz')
)

print(f"(Before) x_train has shape: {x_train.shape}")
print(f"(Before) x_test has shape: {x_test.shape}")

print(f"(Before) x_train has - min: {np.min(x_train)}, max: {np.max(x_train)}")
print(f"(Before) x_test has - min: {np.min(x_test)}, max: {np.max(x_test)}")

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

print(f"(After) x_train has shape: {x_train.shape}")
print(f"(After) x_test has shape: {x_test.shape}")

x_train = (x_train - np.mean(x_train, axis=1, keepdims=True)) / np.std(x_train, axis=1, keepdims=True)
x_test = (x_test - np.mean(x_test, axis=1, keepdims=True)) / np.std(x_test, axis=1, keepdims=True)

print(f"(After) x_train has - min: {np.min(x_train)}, max: {np.max(x_train)}")
print(f"(After) x_test has - min: {np.min(x_test)}, max: {np.max(x_test)}")

11501568/11490434 [==============================] - 7s 1us/step
(Before) x_train has shape: (60000, 28, 28)
(Before) x_test has shape: (10000, 28, 28)
(Before) x_train has - min: 0, max: 255
(Before) x_test has - min: 0, max: 255
(After) x_train has shape: (60000, 784)
(After) x_test has shape: (10000, 784)
(After) x_train has - min: -0.8508020140306101, max: 7.770145734421186
(After) x_test has - min: -0.731905259880747, max: 7.57798901103678


In [42]:
batch_size = 100
img_width, img_height = 28,28
input_size = img_height * img_width
num_labels = 10

def mnist_model():
    """ Defining the model using Keras sequential API """
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(500, activation='relu'),
        tf.keras.layers.Dense(250, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Defining the model, optimizer and a loss function
model = mnist_model()
optimizer = tf.keras.optimizers.RMSprop()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'])

NUM_EPOCHS = 10

""" Model training """    

# Creating onehot encoded labels
y_onehot_train = np.zeros((y_train.shape[0], num_labels), dtype=np.float32)
y_onehot_train[np.arange(y_train.shape[0]), y_train] = 1.0

# Training Phase 
train_history = model.fit(x_train,y_onehot_train, batch_size=batch_size, epochs=NUM_EPOCHS, validation_split=0.2)

""" Testing phase """

# Test inputs and targets
y_onehot_test = np.zeros((y_test.shape[0], num_labels), dtype=np.float32)
y_onehot_test[np.arange(y_test.shape[0]), y_test] = 1.0

# Evaulte on test data
test_res = model.evaluate(x_test, y_onehot_test, batch_size=batch_size)

print("Testing Results: ")
print(f"\tLoss: {test_res[0]}")
print(f"\tAccuracy: {test_res[1]}")

Epoch 1/10
480/480 [==============================] - 8s 14ms/step - loss: 0.2310 - acc: 0.9287 - val_loss: 0.1186 - val_acc: 0.9643
Epoch 2/10
480/480 [==============================] - 7s 15ms/step - loss: 0.0933 - acc: 0.9719 - val_loss: 0.1101 - val_acc: 0.9682
Epoch 3/10
480/480 [==============================] - 6s 12ms/step - loss: 0.0655 - acc: 0.9801 - val_loss: 0.1158 - val_acc: 0.9706
Epoch 4/10
480/480 [==============================] - 6s 12ms/step - loss: 0.0482 - acc: 0.9850 - val_loss: 0.1172 - val_acc: 0.9731
Epoch 5/10
480/480 [==============================] - 6s 13ms/step - loss: 0.0403 - acc: 0.9881 - val_loss: 0.1234 - val_acc: 0.9749
Epoch 6/10
480/480 [==============================] - 7s 15ms/step - loss: 0.0316 - acc: 0.9902 - val_loss: 0.1399 - val_acc: 0.9757
Epoch 7/10
480/480 [==============================] - 6s 13ms/step - loss: 0.0286 - acc: 0.9915 - val_loss: 0.1342 - val_acc: 0.9751
Epoch 8/10
480/480 [==============================] - 6s 13ms/step - 